In [2]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.graph.message import add_messages



In [3]:
class ChatState(TypedDict):
    messages : Annotated[list[BaseMessage],add_messages]

In [4]:
llm = ChatOllama(model="llama3.2:latest")

In [6]:
def chat_node(state:ChatState) :
    # take user query from state
    messages = state['messages']
    #sending it to llm
    response = llm.invoke(messages)
    # response store into the state
    return {'messages': [response]}

In [ ]:
graph = StateGraph(ChatState)

graph.add_node("chat_node", chat_node)

graph.add_edge(START,"chat_node")

#graph.add_edge("chat_node", END)

#graph.compile()

workflow = graph.compile()

print(workflow)

initial_state={
   'messages': HumanMessage(content="What are the differences between agent and agentic ai?")
}

workflow.invoke(initial_state)['messages'][-1].content
